In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from lib import load_curvecurator, plot_fit_type, plot_fit, fit, to_p_space, from_p_space

# Increase plot resolution
plt.rcParams['figure.dpi'] = 200

In [ ]:
palette = sns.color_palette("tab10")
palette = {
    "Target": palette[0],
    "Off-target": palette[1],
    "Both": palette[2],
}

RANGE_DELTA = 1

In [ ]:
drugs = {
    "Afatinib": {
        'identifier': "10184653/2reps_only",
        'target_ec50_nm': 3,
        'target': "EGFR"
    },
    "Lapatinib": {
        "identifier": "208908",
        "target_ec50_nm": 65,
    },
    "PF-3758309": {
        "identifier": "25227462",
        'target_ec50_nm': 119,
        'target': 'PAK4'
    },
    "Dasatinib": {
        "identifier": "3062316",
        'target_ec50_nm': 5,
    },
    "92131101": {
        "identifier": "92131101",
        'target_ec50_nm': 10,
    },
    "Tipifarnib": {
        "identifier": "159324",
        'target_ec50_nm': 40,
    },
    "Docetaxel": {
        "identifier": "9877265",
        'target_ec50_nm': 50,
    },
    "168312962": {
        "identifier": "168312962",
        'target_ec50_nm': 40,
    },
    "Selumetinib": {
        "identifier": "10127622",
        'target_ec50_nm': 200,
    },
    "11364421": {
        "identifier": "11364421",
        'target_ec50_nm': 8,
    },
    "Bleomycin": {
        "identifier": "46930982",
        'target_ec50_nm': 50,
    },
    "Alisertib": {
        "identifier": "24771867",
        'target_ec50_nm': 30,
    },
    "Oligomycin A": {
        "identifier": "6364620",
        'target_ec50_nm': 30,
    },
    "Trametinib": {
        "identifier": "11707110",
        'target_ec50_nm': 20,
    },
    "Leptomycin B": {
        "identifier": "6917907",
        'target_ec50_nm': 30,
    },
    "Vincristine": {
        "identifier": "5388993",
        'target_ec50_nm': 100,
    },
    "Idelalisib": {
        "identifier": "11625818",
        'target_ec50_nm': 80,
    },
    "Rapamycin": {
        "identifier": "5284616",
        'target_ec50_nm': 30,
    },
}

drug = "Afatinib"

df_curvecurator, df_doses = load_curvecurator(drugs[drug]['identifier'])
target = drugs[drug].get('target', None)
target_pec50 = to_p_space(drugs[drug]['target_ec50_nm'] / 1e9)
target_pec50_range = np.array([target_pec50 - RANGE_DELTA, target_pec50 + RANGE_DELTA])
target_ec50_range = from_p_space(target_pec50_range)

df_curvecurator = fit(df_curvecurator, df_doses, target_pec50_range)

In [ ]:
other_drug = "Lapatinib"
df_curvecurator_other, df_doses_other = load_curvecurator(drugs[other_drug]['identifier'])
target_pec50_other = to_p_space(drugs[other_drug]['target_ec50_nm'] / 1e9)
target_pec50_range_other = np.array([target_pec50_other - RANGE_DELTA, target_pec50_other + RANGE_DELTA])
target_ec50_range_other = from_p_space(target_pec50_range_other)

df_curvecurator_other = fit(df_curvecurator_other, df_doses_other, target_pec50_range_other)

In [ ]:
cl_intersection = list(set(df_curvecurator.index).intersection(set(df_curvecurator_other.index)))
column = "double_target_effect_size"

df_intersection = pd.DataFrame({
    drug: df_curvecurator.loc[cl_intersection, column],
    f"{drug}_fit_type": df_curvecurator.loc[cl_intersection, "Double fit type"],
    other_drug: df_curvecurator_other.loc[cl_intersection, column],
    f"{other_drug}_fit_type": df_curvecurator_other.loc[cl_intersection, "Double fit type"],
})

sns.jointplot(data=df_intersection, x=drug, y=other_drug, hue=f"{other_drug}_fit_type", palette=palette)